# Clustering Crypto

In [76]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from pathlib import Path

### Fetching Cryptocurrency Data

In [100]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [361]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
r = requests.get(url)
data = r.json()

In [362]:
list(data.keys())

['Response', 'Message', 'Data', 'RateLimit', 'HasWarning', 'Type']

In [363]:
# Alternatively, use the provided csv file:
file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame
data = data['Data']
crypto_df = pd.DataFrame.from_dict(data)

In [364]:
crypto_df

,42,300,365,404,433,611,808,888,1337,2015,...,QDX,EMPIRE,BCNA,SOUND,DREAMS,CATZ,SOSNOVKINO,TST,SNK,MBX
Id,4321,749869,33639,21227,926547,20909,28223,29462,20824,3744,...,944705,944707,927428,944710,944713,944696,196612,944717,944716,944724
Url,/coins/42/overview,/coins/300/overview,/coins/365/overview,/coins/404/overview,/coins/433/overview,/coins/611/overview,/coins/808/overview,/coins/888/overview,/coins/1337/overview,/coins/2015/overview,...,/coins/qdx/overview,/coins/empire/overview,/coins/bcna/overview,/coins/sound/overview,/coins/dreams/overview,/coins/catz/overview,/coins/sosnovkino/overview,/coins/tst/overview,/coins/snk/overview,/coins/mbx/overview
ImageUrl,/media/35650717/42.jpg,/media/27010595/300.png,/media/352070/365.png,/media/35650851/404-300x300.jpg,/media/34836095/433.png,/media/35650940/611-sixeleven.png,/media/351513/808.png,/media/351639/888.png,/media/35520987/elite.png,/media/20180/2015.png,...,/media/39108992/qdx.png,/media/39108993/empire.png,/media/39108994/bcna.png,/media/39108995/sound.png,/media/39108997/dreams.png,/media/39108988/catz.png,/media/1383865/snk.png,/media/39108998/tst.png,/media/39108999/snk.png,/media/39109004/mbx.png
ContentCreatedOn,1427211129,1517935016,1480032918,1466100361,1541597321,1465914773,1473980395,1475534352,1465838687,1425316878,...,1637245576,1637246468,1543411914,1637248574,1637249613,1637242143,1501035708,1637251491,1637251368,1637254217
Name,42,300,365,404,433,611,808,888,1337,2015,...,QDX,EMPIRE,BCNA,SOUND,DREAMS,CATZ,SOSNOVKINO,TST,SNK,MBX
Symbol,42,300,365,404,433,611,808,888,1337,2015,...,QDX,EMPIRE,BCNA,SOUND,DREAMS,CATZ,SOSNOVKINO,TST,SNK,MBX
CoinName,42 Coin,300 token,365Coin,404Coin,433 Token,SixEleven,808,Octocoin,EliteCoin,2015 coin,...,Quidax,Empire Token,BitCanna,Sound Coin,Dreams Quest,CatzCoin,Sosnovkino,Standard Token,Snake Token,MobieCoin
FullName,42 Coin (42),300 token (300),365Coin (365),404Coin (404),433 Token (433),SixEleven (611),808 (808),Octocoin (888),EliteCoin (1337),2015 coin (2015),...,Quidax (QDX),Empire Token (EMPIRE),BitCanna (BCNA),Sound Coin (SOUND),Dreams Quest (DREAMS),CatzCoin (CATZ),Sosnovkino (SOSNOVKINO),Standard Token (TST),Snake Token (SNK),MobieCoin (MBX)
Description,Everything about 42 coin is 42 - apart from th...,300 token is an ERC20 token. This Token was cr...,365Coin is a Proof of Work and Proof of Stake ...,404 is a PoW/PoS hybrid cryptocurrency that al...,433 Token is a decentralised soccer platform t...,"611 is a Namecoin based cryptocurrency, and it...",808 is a coin develop for the music community ...,OCTO is a Counterparty asset with focus on dec...,1337 coin was created as an experimental Proof...,Twenty15 Coin was conceived to be an asset bac...,...,"As the native token of Quidax, QDX powers key ...",Empire Token is a decentralized token on the B...,BitCanna will provide a decentralized payment ...,Sound Coin is a full-on-chain liquidity protoc...,Dreams Quest is building a metaverse-based eco...,Launched on 1 May 2021 by a team based in Aust...,Sosnovkino is a decentralized land development...,The Standard Protocol is a decentralized finan...,Cryptosnake is an incremental money-making gam...,MobiePay is a way for both consumers and merch...
AssetTokenStatus,N/A,N/A,N/A,N/A,Finished,N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,Finished,N/A,N/A,N/A,Finished,Upcoming,N/A,N/A


In [365]:
coin_supply_df = pd.read_csv(file_path)
coin_supply_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [366]:
coin_supply_df = coin_supply_df.set_index('CoinName')
coin_supply_df.head()

,Unnamed: 0,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
CoinName,,,,,,
42 Coin,42,Scrypt,True,PoW/PoS,4.199995e+01,42
365Coin,365,X11,True,PoW/PoS,NaN,2300000000
404Coin,404,Scrypt,True,PoW/PoS,1.055185e+09,532000000
SixEleven,611,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [367]:
coin_supply_df.drop(columns=[
    'Unnamed: 0',
    'Algorithm',
    'IsTrading',
    'ProofType',
], inplace=True)
coin_supply_df.head()

,TotalCoinsMined,TotalCoinSupply
CoinName,,
42 Coin,4.199995e+01,42
365Coin,NaN,2300000000
404Coin,1.055185e+09,532000000
SixEleven,NaN,611000
808,0.000000e+00,0


TotalCoinSupply was missing from the website.

In [368]:
crypto_df = crypto_df.T
crypto_df.head()

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,MaxSupply,MktCapPenalty,IsUsedInDefi,IsUsedInNft,PlatformType,BuiltOn,SmartContractAddress,DecimalPoints,Difficulty,AlgorithmType
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,300,0,0,0,token,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18,NaN,NaN
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [369]:
crypto_df = crypto_df.set_index('CoinName')
crypto_df.head()

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,FullName,Description,AssetTokenStatus,Algorithm,...,MaxSupply,MktCapPenalty,IsUsedInDefi,IsUsedInNft,PlatformType,BuiltOn,SmartContractAddress,DecimalPoints,Difficulty,AlgorithmType
CoinName,,,,,,,,,,,,,,,,,,,,,
42 Coin,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,Scrypt,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
300 token,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,N/A,...,300,0,0,0,token,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18,NaN,NaN
365Coin,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,X11,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
404Coin,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,Scrypt,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
433 Token,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,N/A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Preprocessing

In [370]:
crypto_df.columns

Index(['Id', 'Url', 'ImageUrl', 'ContentCreatedOn', 'Name', 'Symbol',
       'FullName', 'Description', 'AssetTokenStatus', 'Algorithm', 'ProofType',
       'SortOrder', 'Sponsored', 'Taxonomy', 'Rating', 'IsTrading',
       'TotalCoinsMined', 'CirculatingSupply', 'BlockNumber',
       'NetHashesPerSecond', 'BlockReward', 'BlockTime', 'AssetLaunchDate',
       'AssetWhitepaperUrl', 'AssetWebsiteUrl', 'MaxSupply', 'MktCapPenalty',
       'IsUsedInDefi', 'IsUsedInNft', 'PlatformType', 'BuiltOn',
       'SmartContractAddress', 'DecimalPoints', 'Difficulty', 'AlgorithmType'],
      dtype='object')

In [371]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df.drop(columns = [
    'AlgorithmType',
    'AssetLaunchDate',
    'AssetTokenStatus',
    'AssetWebsiteUrl',
    'AssetWhitepaperUrl',
    'BlockNumber',
    'BlockReward',
    'BlockTime',
    'BuiltOn',
    'CirculatingSupply',
    'ContentCreatedOn',
    'DecimalPoints',
    'Description',
    'Difficulty',
    'FullName',
    'Id',
    'ImageUrl',
    'IsUsedInDefi',
    'IsUsedInNft',
    'MaxSupply',
    'MktCapPenalty',
    'Name',
    'NetHashesPerSecond',
    'PlatformType',
    'Rating',
    'SmartContractAddress',
    'SortOrder',
    'Sponsored',
    'Taxonomy',
    'TotalCoinsMined',
    'Url'
], inplace = True)


In [372]:
crypto_df.columns

Index(['Symbol', 'Algorithm', 'ProofType', 'IsTrading'], dtype='object')

Need to join TotalCoinSupply to crypto_df

In [373]:
crypto_df

,Symbol,Algorithm,ProofType,IsTrading
CoinName,,,,
42 Coin,42,Scrypt,PoW/PoS,True
300 token,300,N/A,N/A,True
365Coin,365,X11,PoW/PoS,True
404Coin,404,Scrypt,PoW/PoS,True
433 Token,433,N/A,N/A,False
...,...,...,...,...
CatzCoin,CATZ,N/A,N/A,False
Sosnovkino,SOSNOVKINO,N/A,N/A,False
Standard Token,TST,N/A,N/A,False


In [374]:
coin_supply_df

,TotalCoinsMined,TotalCoinSupply
CoinName,,
42 Coin,4.199995e+01,42
365Coin,NaN,2300000000
404Coin,1.055185e+09,532000000
SixEleven,NaN,611000
808,0.000000e+00,0
...,...,...
BitcoinPlus,1.283270e+05,1000000
DivotyCoin,2.149121e+07,100000000
Giotto Coin,NaN,233100000


In [375]:
joined_df = pd.merge(crypto_df, coin_supply_df, on='CoinName', how='outer')
joined_df

,Symbol,Algorithm,ProofType,IsTrading,TotalCoinsMined,TotalCoinSupply
CoinName,,,,,,
42 Coin,42,Scrypt,PoW/PoS,True,4.199995e+01,42
300 token,300,N/A,N/A,True,NaN,NaN
365Coin,365,X11,PoW/PoS,True,NaN,2300000000
404Coin,404,Scrypt,PoW/PoS,True,1.055185e+09,532000000
433 Token,433,N/A,N/A,False,NaN,NaN
...,...,...,...,...,...,...
ClubCoin,NaN,NaN,NaN,NaN,1.036219e+08,160000000
Radium,NaN,NaN,NaN,NaN,3.821246e+06,9000000
Creditbit,NaN,NaN,NaN,NaN,1.690102e+07,16504333


In [376]:
crypto_df = joined_df.copy()
crypto_df

,Symbol,Algorithm,ProofType,IsTrading,TotalCoinsMined,TotalCoinSupply
CoinName,,,,,,
42 Coin,42,Scrypt,PoW/PoS,True,4.199995e+01,42
300 token,300,N/A,N/A,True,NaN,NaN
365Coin,365,X11,PoW/PoS,True,NaN,2300000000
404Coin,404,Scrypt,PoW/PoS,True,1.055185e+09,532000000
433 Token,433,N/A,N/A,False,NaN,NaN
...,...,...,...,...,...,...
ClubCoin,NaN,NaN,NaN,NaN,1.036219e+08,160000000
Radium,NaN,NaN,NaN,NaN,3.821246e+06,9000000
Creditbit,NaN,NaN,NaN,NaN,1.690102e+07,16504333


In [377]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df.loc[crypto_df['IsTrading'] == True]

In [378]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df.loc[crypto_df['Algorithm'] != 'N/A']

In [379]:
# Remove the "IsTrading" column
crypto_df.drop(columns = ['IsTrading'], inplace = True)

In [380]:
# Remove rows with at least 1 null value
crypto_df.isnull().sum()

Symbol               0
Algorithm            0
ProofType            0
TotalCoinsMined    991
TotalCoinSupply    536
dtype: int64

In [381]:
crypto_df = crypto_df.dropna()
crypto_df.isnull().sum()

Symbol             0
Algorithm          0
ProofType          0
TotalCoinsMined    0
TotalCoinSupply    0
dtype: int64

In [382]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df.loc[crypto_df['TotalCoinsMined'] != 0]

In [383]:
crypto_df = crypto_df.loc[crypto_df['TotalCoinSupply'] != 0]

In [384]:
# Drop rows where there are 'N/A' text values
crypto_df = crypto_df[crypto_df!='N/A']

In [385]:
crypto_df.reset_index(inplace=True)
crypto_df

,CoinName,Symbol,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,42,Scrypt,PoW/PoS,4.199995e+01,42
1,404Coin,404,Scrypt,PoW/PoS,1.055185e+09,532000000
2,EliteCoin,1337,X13,PoW/PoS,2.927942e+10,314159265359
3,BitcoinDark,BTCD,SHA-256,PoW/PoS,1.288862e+06,22000000
4,PayCoin,XPY,SHA-256,PoS,1.199533e+07,12500000
...,...,...,...,...,...,...
505,MegaCoin,MEC,Scrypt,PoW,3.815200e+07,42000000
506,ZCash,ZEC,Equihash,PoW,7.383056e+06,21000000
507,Oxycoin,OXYC,DPoS,DPoS,1.122382e+09,0
508,PAXEX,PAXEX,X11,PoS,4.504639e+06,100000000


In [387]:
crypto_df = crypto_df.loc[crypto_df['TotalCoinSupply'] != '0']
crypto_df

,CoinName,Symbol,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,42,Scrypt,PoW/PoS,4.199995e+01,42
1,404Coin,404,Scrypt,PoW/PoS,1.055185e+09,532000000
2,EliteCoin,1337,X13,PoW/PoS,2.927942e+10,314159265359
3,BitcoinDark,BTCD,SHA-256,PoW/PoS,1.288862e+06,22000000
4,PayCoin,XPY,SHA-256,PoS,1.199533e+07,12500000
...,...,...,...,...,...,...
504,MonaCoin,MONA,Scrypt,PoW,6.819467e+07,105120000
505,MegaCoin,MEC,Scrypt,PoW,3.815200e+07,42000000
506,ZCash,ZEC,Equihash,PoW,7.383056e+06,21000000
508,PAXEX,PAXEX,X11,PoS,4.504639e+06,100000000


In [388]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coinname_df = pd.DataFrame(
    data = crypto_df, columns = ['CoinName']
)
coinname_df.head()

,CoinName
0,42 Coin
1,404Coin
2,EliteCoin
3,BitcoinDark
4,PayCoin


In [389]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
#crypto_df.reset_index(inplace=True)
crypto_df = crypto_df.drop(columns = 'CoinName')

In [390]:
crypto_df.sample(30)

,Symbol,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
254,KRONE,Scrypt,PoW,1.745375e+07,84000000
255,ACC,Scrypt,PoW,3.264673e+07,100000000
135,XRA,X11,PoW/PoS,1.345549e+08,75000000
458,ONT,VBFT,PoS,6.508486e+08,1000000000
115,EDRC,SHA-256,PoW/PoS,3.669692e+06,22000000
319,CIF,X11,PoW,2.112976e+08,500000000
220,GRW,Scrypt,PoS,2.951355e+08,2000000000
113,RVR,Scrypt,PoW,2.100000e+08,210000000
444,BBP,Proof-of-BibleHash,POBh,1.772092e+09,5200000000
38,NOBL,Scrypt,PoW,2.366066e+09,15000000000


In [391]:
crypto_df = crypto_df.set_index('Symbol')
crypto_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Symbol,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTCD,SHA-256,PoW/PoS,1.288862e+06,22000000
XPY,SHA-256,PoS,1.199533e+07,12500000
...,...,...,...,...
MONA,Scrypt,PoW,6.819467e+07,105120000
MEC,Scrypt,PoW,3.815200e+07,42000000
ZEC,Equihash,PoW,7.383056e+06,21000000


In [392]:
# Create dummy variables for text features
crypto_dummies= pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'])
crypto_dummies.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,Algorithm_Cloverhash,...,ProofType_PoW,ProofType_PoW/Hive,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoSe,ProofType_PoW/nPoS,ProofType_Pos,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
Symbol,,,,,,,,,,,,,,,,,,,,,
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
BTCD,1.288862e+06,22000000,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
XPY,1.199533e+07,12500000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [393]:
# Standardize data
crypto_scaled = StandardScaler().fit_transform(crypto_dummies)
print(crypto_scaled[0:1])

[[-0.10703153 -0.15174782 -0.04588315 -0.04588315 -0.06495698 -0.07963978
  -0.04588315 -0.04588315 -0.06495698 -0.04588315 -0.04588315 -0.19824558
  -0.04588315 -0.07963978 -0.04588315 -0.09205746 -0.06495698 -0.06495698
  -0.04588315 -0.16081688 -0.04588315 -0.04588315 -0.12216944 -0.04588315
  -0.04588315 -0.07963978 -0.04588315 -0.04588315 -0.04588315 -0.04588315
  -0.04588315 -0.04588315 -0.04588315 -0.07963978 -0.10303257 -0.07963978
  -0.04588315 -0.13074409 -0.13882344 -0.14648968 -0.04588315 -0.06495698
  -0.04588315 -0.04588315 -0.07963978 -0.17407766 -0.04588315 -0.04588315
  -0.04588315 -0.06495698 -0.16756418 -0.04588315 -0.327073   -0.04588315
  -0.09205746 -0.07963978 -0.06495698 -0.04588315  1.40534693 -0.04588315
  -0.06495698 -0.04588315 -0.04588315 -0.04588315 -0.04588315 -0.04588315
  -0.04588315 -0.38339008 -0.04588315 -0.17407766 -0.04588315 -0.09205746
  -0.09205746 -0.12216944 -0.04588315 -0.04588315 -0.04588315 -0.12216944
  -0.04588315 -0.04588315 -0.04588315 

In [394]:
crypto_df = crypto_df[['Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined']]
crypto_df

,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined
Symbol,,,,
42,Scrypt,PoW/PoS,42,4.199995e+01
404,Scrypt,PoW/PoS,532000000,1.055185e+09
1337,X13,PoW/PoS,314159265359,2.927942e+10
BTCD,SHA-256,PoW/PoS,22000000,1.288862e+06
XPY,SHA-256,PoS,12500000,1.199533e+07
...,...,...,...,...
MONA,Scrypt,PoW,105120000,6.819467e+07
MEC,Scrypt,PoW,42000000,3.815200e+07
ZEC,Equihash,PoW,21000000,7.383056e+06


### Reducing Dimensions Using PCA

In [395]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)

crypto_pca = pca.fit_transform(crypto_scaled)

In [396]:
# Create a DataFrame with the principal components data
pca_df = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"]
)
pca_df.head()

,PC 1,PC 2,PC 3
0,-0.330781,-1.011356,-0.604393
1,-0.315091,-1.011360,-0.604765
2,2.168395,-1.614140,-0.670696
3,-0.338038,-0.936652,-0.431472
4,-0.280244,-0.642230,-0.064627


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [397]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(crypto_pca)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

C:\Users\matth\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.



:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [360]:
# Initialize the K-Means model
model = KMeans(n_clusters=5, random_state=0)
# Fit the model
model.fit(pca_df)
# Predict clusters
predictions = model.predict(pca_df)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
pca_df['class'] = model.labels_

df_merged = crypto_df.join(coinname_df, how='inner')
df_merged = df_merged.append(pca_df)

df_merged#May need to reorder columns

,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,CoinName,PC 1,PC 2,PC 3,class
0,NaN,NaN,NaN,NaN,NaN,-0.346303,-1.110628,-0.387139,1.0
1,NaN,NaN,NaN,NaN,NaN,-0.330104,-1.110629,-0.387461,1.0
2,NaN,NaN,NaN,NaN,NaN,2.236067,-1.692972,-0.411193,4.0
3,NaN,NaN,NaN,NaN,NaN,-0.360600,-0.871318,-0.303668,1.0
4,NaN,NaN,NaN,NaN,NaN,-0.299240,-0.465189,-0.020334,1.0
...,...,...,...,...,...,...,...,...,...
505,NaN,NaN,NaN,NaN,NaN,-0.141866,1.116456,-0.046289,0.0
506,NaN,NaN,NaN,NaN,NaN,-0.092049,2.023445,0.264701,0.0
507,NaN,NaN,NaN,NaN,NaN,4.067594,-2.366595,0.248922,4.0
508,NaN,NaN,NaN,NaN,NaN,-0.354211,-0.843180,-0.104248,1.0


### Visualizing Results

#### 3D-Scatter with Clusters

In [320]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    df_merged,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    hover_name='CoinName',
    hover_data= ['Algorithm'],
    color="class",
    symbol="class",
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

#### Table of Tradable Cryptocurrencies

In [321]:
# Table with tradable cryptos
columns = ['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'class']

In [322]:
# Print the total number of tradable cryptocurrencies
df_merged.hvplot.table(columns)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

#### Scatter Plot with Tradable Cryptocurrencies

In [323]:
# Scale data to create the scatter plot
df_merged['TotalCoinsMined'] = df_merged['TotalCoinsMined'].astype(float) / 1000000
df_merged['TotalCoinSupply'] = df_merged['TotalCoinSupply'].astype(float) / 1000000

In [324]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
df_merged.hvplot(
    kind="scatter", 
    x="TotalCoinsMined", 
    y="TotalCoinSupply", 
    c='class', 
    colormap="viridis", 
    hover_cols=['CoinName']
)

:Scatter   [TotalCoinsMined]   (TotalCoinSupply,class,CoinName)